### Faster RCNN Installation
* In production, we want to install the custom faster rcnn without cloning the whole source repository. This is an example of how to do that.

In [ ]:
# https://adamj.eu/tech/2019/03/11/pip-install-from-a-git-repository/
# Install customized linc-detector faster-rcnn
# !pip install --upgrade --force-reinstall git+https://github.com/linc-lion/LINC-detector.git@ba36a5bfa5ba7b9035977c02b1d8ed253f074e8d

### Working Directory
* The jupyter notebook is in the same repository as the source. So, we are switching to the parent directory as working directory so that imports work.
* Do the following only for local development. 

In [2]:
# Get parent directory
if 'parent_dir' not in globals():
    import os
    current_dir = os.getcwd()
    parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
parent_dir

'/Users/sglee/work/git/linc/LINC-detector'

In [3]:
cd $parent_dir

/Users/sglee/work/git/linc/LINC-detector


### Requirements installation

In [ ]:
pip install -r requirements.txt

In [ ]:
# Install pycocotools
!pip install --upgrade --force-reinstall cython
!pip install --upgrade --force-reinstall -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install setuptools==59.5.0 # https://github.com/pytorch/pytorch/issues/69894

### Create training datasets

In [ ]:
# !python datasets/create_all_but_ws_and_fb_dataset.py

### Resolve SageMaker specific bug (optional if not run on SageMaker)
* https://github.com/aws/amazon-sagemaker-examples/issues/3154

In [ ]:
# %%capture
# import IPython
# import sys

# !{sys.executable} -m pip install ipywidgets
# IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

### Imports

In [4]:
import datetime
import os
import time
import shutil
import subprocess
import sys

import torch
import torch.utils.data
from linc.detector.models import detection
from torch.utils.tensorboard import SummaryWriter

from linc.detector.helper.coco_utils import get_coco  # get_coco_kp
from linc.detector.helper.group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
from linc.detector.helper.engine import train_one_epoch, evaluate
from linc.detector.helper import utils

%load_ext tensorboard

### Model Training
* The base training code is extracted from [pytorch example](https://github.com/pytorch/vision/blob/528651a031a08f9f97cc75bd619a326387708219/references/detection/train.py)

In [5]:
# Path to COCO formatted object detection dataset
data_path = 'datasets/coco_all_but_ws_and_fb/'  

# Ignorable arguments
epochs = 2 # 35
save_every_num_epochs = None  # Optional
evaluate_every_num_epochs = 2
lr = 0.01
momentum = 0.9
weight_decay = 1e-4
lr_steps = [10, 11]
lr_gamma = 0.1
batch_size = 3
workers = 1
run_name = None  # Optional, str used to name Tensorboard summaries
num_draw_predictions = 5
draw_threshold = 0.5
aspect_ratio_group_factor = 0

In [6]:
# Training code is based on 

print("Create summary writer for Tensorboard")
if run_name:
    log_dir_path = f"{run_name}" if run_name else None
    if os.path.isdir(log_dir_path):
        delete = input(f"Summary folder '{log_dir_path}' already exists. Overwrite it [yes, y / no, n]?")
        if delete in ('yes', 'y'):
            shutil.rmtree(log_dir_path)
        else:
            print(f"Chose another run name or delete the folder then!")
            exit()
else:
    log_dir_path = None
writer = SummaryWriter(log_dir=log_dir_path)

# Add some useful text summaries (Tensorboard uses markdown to render text).
# writer.add_text('Command executed', f"python {' '.join(sys.argv)}")
# writer.add_text('Arguments', str(args).replace(", ", ",  \n").replace("Namespace(", "").replace(")", ""))

print("Create datasets")
dataset, num_classes, label_names = get_coco(data_path, image_set='train')
print(f"Categorizing into {num_classes} classes")
dataset_test, _, _ = get_coco(data_path, image_set='val')

print("Create samplers")
train_sampler = torch.utils.data.RandomSampler(dataset)
test_sampler = torch.utils.data.SequentialSampler(dataset_test)
group_ids = create_aspect_ratio_groups(dataset, k=aspect_ratio_group_factor)
train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)

print("Create dataloaders")
data_loader = torch.utils.data.DataLoader(dataset, 
                                          batch_sampler=train_batch_sampler, 
                                          num_workers=workers, 
                                          collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(dataset_test, 
                                               batch_size=1,
                                               sampler=test_sampler, 
                                               num_workers=workers, 
                                               collate_fn=utils.collate_fn)

print("Create model")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
model = detection.__dict__['fasterrcnn_resnet50_fpn'](num_classes=num_classes, pretrained=False)
model.to(device)
model_without_ddp = model

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, 
                            lr=lr, 
                            momentum=momentum, 
                            weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                    milestones=lr_steps, 
                                                    gamma=lr_gamma)

print("Start training")
start_time = time.time()
for epoch in range(epochs):
    start_epoch = time.time()
    train_one_epoch(
        model, optimizer, data_loader, device, epoch, 20, writer, label_names
    )
    print(f"Epoch time {time.time() - start_epoch}")
    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step=epoch)
    lr_scheduler.step()

    if save_every_num_epochs and epoch % save_every_num_epochs == 0:
        utils.save_on_master({
            'model': model_without_ddp.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'args': args,
            'label_names': label_names},
            os.path.join(writer.log_dir, 'model_{}.pth'.format(epoch))
        )

    if epoch % evaluate_every_num_epochs == 0:
        evaluate(
            model, data_loader_test, epoch, writer, draw_threshold,
            label_names, num_draw_predictions, device=device
        )

# Save after training is done
utils.save_on_master({
    'model': model_without_ddp.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_scheduler': lr_scheduler.state_dict(),
    'label_names': label_names},
    os.path.join(writer.log_dir, 'model_finished.pth')
)

writer.close()
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))


Create summary writer for Tensorboard


/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Create datasets
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Categorizing into 32 classes
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Create samplers
Using [0, 1.0, inf] as bins for aspect ratio quantization
Count of instances per bin: [15  1]
Create dataloaders
Create model
Device: cuda
Start training


/Users/sglee/anaconda3/envs/linc-detector-jupyter/lib/python3.8/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
/Users/sglee/anaconda3/envs/linc-detector-jupyter/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [0/5]  eta: 0:00:13  lr: 0.002507  loss: 4.4966 (4.4966)  loss_classifier: 3.6843 (3.6843)  loss_box_reg: 0.0584 (0.0584)  loss_objectness: 0.6997 (0.6997)  loss_rpn_box_reg: 0.0541 (0.0541)  time: 2.7399  data: 1.5254  max mem: 0
Epoch: [0]  [4/5]  eta: 0:00:02  lr: 0.010000  loss: 4.2149 (3.6820)  loss_classifier: 3.3083 (2.8363)  loss_box_reg: 0.0547 (0.0588)  loss_objectness: 0.6951 (0.6905)  loss_rpn_box_reg: 0.0541 (0.0964)  time: 2.4498  data: 1.4385  max mem: 0
Epoch: [0] Total time: 0:00:12 (2.4576 s / it)
Epoch time 12.374770641326904
Test:  [0/5]  eta: 0:00:03  model_time: 0.1116 (0.1116)  evaluator_time: 0.0013 (0.0013)  time: 0.6786  data: 0.5214  max mem: 0
Test:  [4/5]  eta: 0:00:00  model_time: 0.1116 (0.1111)  evaluator_time: 0.0008 (0.0008)  time: 0.6877  data: 0.4991  max mem: 0
Test: Total time: 0:00:03 (0.6951 s / it)
Averaged stats: model_time: 0.1116 (0.1111)  evaluator_time: 0.0008 (0.0008)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metri

In [ ]:
%tensorboard --logdir "./runs"